### Compute Reduction in Features

In [4]:
MONGO_COLLECTION = "CB_RE-RANKER_HYPER_PARAM_VD"

In [5]:
# for r in rows[-17:]:
#     print(str(r["asof"]))

In [6]:
import pandas as pd
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

In [7]:
df = pd.DataFrame(results)
df.tail(3)

,C,accuracy,asof,best_max_parses,best_max_upd,best_min_prob,best_top_n,data_points,early_stopping_iters,extractors,...,max_epochs,max_update_items,min_feat_freq,num_codes,num_feats_MEAN,num_feats_per_fold,num_tags,pa_type,precision,recall
644,0.1,0.983638,2019-06-17 12:54:16.227000,300,NaN,0.0,5,82082.0,3,"[CREL_, Prob-, CChainStats-]",...,3,1,1,2740.0,3559.8,"[3500, 3425, 3440, 3727, 3707]",91,1,0.802250,0.676642
645,0.1,0.983200,2019-06-17 13:06:22.090000,300,NaN,0.0,5,82082.0,3,"[CREL_, Prob-, CChainStats-]",...,3,1,1,2740.0,3559.8,"[3500, 3425, 3440, 3727, 3707]",91,2,0.795741,0.668248
646,0.1,0.982627,2019-06-17 13:18:35.664000,300,NaN,0.0,5,82082.0,3,"[CREL_, Prob-, CChainStats-]",...,3,1,1,2740.0,3559.8,"[3500, 3425, 3440, 3727, 3707]",91,2,0.787150,0.657299


In [17]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", 
    # "best_min_prob", "extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type"]
df[cols].head(5)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
195,0.746779,0.789110,0.708759,2019-06-14 01:50:35.450000,2,0.0100,300,1,0.01,ml,1
319,0.745815,0.788767,0.707299,2019-06-15 00:06:57.168000,2,0.0025,300,10,0.01,pb,1
39,0.745737,0.784994,0.710219,2019-06-13 19:35:28.095000,2,0.0025,300,2,0.01,ml,1
163,0.745731,0.776680,0.717153,2019-06-14 00:39:56.654000,2,0.0025,300,1,0.01,ml,1
175,0.745638,0.795285,0.701825,2019-06-14 01:06:17.901000,2,0.0025,300,10,0.01,ml,1


In [18]:
# Best ones that are not 2
df[df.best_top_n != 2][cols].head()

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
477,0.744212,0.768364,0.721533,2019-06-16 12:42:58.688000,1,0.0005,300,2,0.01,ml,1
475,0.743945,0.785946,0.706204,2019-06-16 12:22:39.787000,1,0.0005,300,2,0.00,ml,1
485,0.743789,0.774181,0.715693,2019-06-16 14:02:16.136000,1,0.0005,300,5,0.01,ml,1
483,0.743748,0.786412,0.705474,2019-06-16 13:41:53.776000,1,0.0005,300,5,0.00,ml,1
640,0.743689,0.794606,0.698905,2019-06-17 12:06:13.169000,5,0.0100,300,1,0.01,ml,1
